In [59]:
import numpy as np
import pandas as pd
import openpyxl
import os
import win32com.client

from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

L'objectif de ce script est de générer des fichiers Excel comportant 4 TCD dans 4 onglets différents à partir de fichier csv nichés dans des sous-dossiers d'un dossier principal à parcourrir.

L'idée est de copier/coller les information du CSV dans un template préalablement préparé (fichier_destination.xlsx) de manière à n'avoir qu'à actualiser les sources de TCD à l'ouverture de chaque fichier final.

Les fichiers finaux doivent être renommé de la manière suivante :
- code du logiciel de destination (ici Sage 1000) : correspondant au 3 premiers caractères du code présent dans la première colonne
- un chaine de caractères fixes
- le numéro d'ordre du fichier dans le sous-dossier

L'ensemble des fichiers générés doivent être stocké dans un dossier séparé.

Une table de synthèse doit être générée contenant:
- le code logiciel source : correspondant au nom du sous-dossier
- le code du logiciel de destination : cf ci-dessus
- le nom du fichier source
- le nom du fichier final

## Chargement du fichier de destination

In [60]:
# grâce à openpyxl on charge le fichier de destination. Celui-ci contient 5 onglets. Nous allons travailler 
#sur la feuille source

wb_destination = load_workbook("fichier_destination.xlsx")
wb_destination.sheetnames

['Sources', 'TCD Cohérence', 'TCD BG', 'TCD BGAUX', 'TCD BANA']

In [61]:
sheet = wb_destination["Sources"]


## Exploration rapide des répertoires et fichiers à traiter

In [62]:
# le dossier source contient 23 sous-dossiers 
folderpath = "dossier source"
subfolderlist = os.listdir(folderpath)
len(subfolderlist)

23

In [5]:
# chaque sous dossier contient 1 ou 2 csv. Au final nous devons donc générer 27 fichiers Excel
list = []
for subfolder in subfolderlist:
    filelist = os.listdir("{}\{}".format(folderpath,subfolder))
    list.append(len(filelist))
print (list)
print (sum(list))

[1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]
27


In [6]:
# l'encodage de mes csv est un peu spécifique donc je spécifie l'encofage, le séparateur et le décimal. 
# Je vérifie notamment que es colonnes "MONTANT XXX" et "M_Signe_XX" sont bien en float
df = pd.read_csv("dossier source\SCT_A\ZZ-MATRICE-21- SCT_A POUR CORRECTION 2021.csv", 
                 encoding="iso8859_15", sep=";", decimal =",")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ETAB                6 non-null      int64  
 1   JNAL                6 non-null      object 
 2   TYPE PIECE          6 non-null      object 
 3   REF PIÈCE           6 non-null      object 
 4   DATE PIÈCE          6 non-null      object 
 5   LIB PIÈCE           6 non-null      object 
 6   CPT GENERAL         6 non-null      int64  
 7   AUXILIAIRE TIERS    1 non-null      float64
 8   LIBELLE ECR         6 non-null      object 
 9   SENS                6 non-null      object 
 10  MONTANT TENUE CPTE  6 non-null      float64
 11  MONTANT TRANSAC     6 non-null      float64
 12  DEV                 6 non-null      object 
 13  DATE ECH            0 non-null      float64
 14  MODE REG            0 non-null      float64
 15  SECTION ANA         2 non-null      object 
 16  DEV TC      

## Construction des fonctions

In [11]:
# A chaque itération je vais devoir effacer la feuille source
# je conserve néanmoins les entêtes

def delete_sheet(sheet):
    while(sheet.max_row>1):
        sheet.delete_rows(2)
    return


In [12]:
delete_sheet(sheet)

In [13]:
wb_destination.save("test.xlsx")

## Construction de la boucle pour changer les sources

In [11]:
# initialisation de la table de synthèse
synthese = pd.DataFrame.from_dict({"NomLogSource": [np.nan], "NomLogCible": [
                                  np.nan], "NomFichierOrigine": [np.nan], "NomFichierFinal": [np.nan]})
# initialisation de l'index du DF synthese
index = 0
# designation du dossier sur lequel itérer
folderpath = "dossier source"

# boucle
for subfolder in subfolderlist:
    #  le nom de sous-dossier est le nom de la société dans mon logiel source 
    nom_log_source = subfolder
    filelist = os.listdir("{}\{}".format(folderpath,subfolder))
    ordre_fichier = 1
    for file in filelist:
        #attention à l'encodage de mes csv notamment au séparateur de décimal pour ne pas générer d'erreur en sortie
        df = pd.read_csv("{}\{}\{}".format(folderpath,subfolder,file), 
                 encoding="iso8859_15", sep=";", decimal =",")
        
        # les 3 premiers caractère de la premiere entrée du df sont le code société dans mon logiciel cible
        nom_log_cible = str(df.loc[0][df.columns[0]])[:3]
        # effacer la feuille "source" du workbook de destination
        delete_sheet(sheet)
        # Remplacer les valeurs par celles du df
        for row in dataframe_to_rows(df, index=False,header=False):
            sheet.append(row)
        # on génére le nom de fichier final
        nom_fichier_final = "{} - chaine de caratere - {}.xlsx".format(nom_log_cible, ordre_fichier)
        ordre_fichier = ordre_fichier + 1
        # on rajoute un rang à la table de synthèse
        synthese.loc[index] = [nom_log_source, nom_log_cible, file, nom_fichier_final]
        index = index+1
        # on sauvegarde le workbook comme un nouveau fichier excel
        wb_destination.save("dossier_final\{}".format(nom_fichier_final))
       
    synthese.to_excel("synthese.xlsx", index=False)

## Actualisation des TCD

In [66]:
dirpath = "dossier_final"
fileList = os.listdir("dossier_final")[21:]
for file in fileList:
    rel_path = "{}\{}".format(dirpath,file)
    abs_path = os.path.abspath(rel_path)

    excel = win32com.client.Dispatch("Excel.Application")

    wb = excel.Workbooks.Open(abs_path) # work only with absolute path
    count = wb.Sheets.Count
    for i in range(2,count):
        ws = wb.Worksheets[i]
        ws.Unprotect() # IF protected

        pivotCount = ws.PivotTables().Count
        for j in range(1, pivotCount+1):
            ws.PivotTables(j).PivotCache().Refresh()

    wb.Close(True) 
    excel.quit()
    excel = None
    del excel